# DDPM Implementation from Scratch

This notebook implements a complete Denoising Diffusion Probabilistic Model (DDPM) from scratch using PyTorch:

1. U-Net architecture for noise prediction
2. Training loop with proper loss computation
3. DDPM and DDIM sampling algorithms
4. Visualization of the denoising process

Based on: [Ho et al., 2020 - Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import math
import warnings
warnings.filterwarnings('ignore')

# Set seeds
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Noise Schedule

The noise schedule defines how much noise is added at each timestep.

In [ ]:
class GaussianDiffusion:
    """Gaussian diffusion process with configurable noise schedule."""
    
    def __init__(self, num_timesteps=1000, beta_schedule='cosine', beta_start=1e-4, beta_end=0.02):
        self.num_timesteps = num_timesteps
        
        # Create beta schedule
        if beta_schedule == 'linear':
            betas = torch.linspace(beta_start, beta_end, num_timesteps)
        elif beta_schedule == 'cosine':
            betas = self._cosine_beta_schedule(num_timesteps)
        elif beta_schedule == 'quadratic':
            betas = torch.linspace(beta_start**0.5, beta_end**0.5, num_timesteps) ** 2
        else:
            raise ValueError(f"Unknown schedule: {beta_schedule}")
        
        self.betas = betas
        self.alphas = 1.0 - betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        self.alphas_cumprod_prev = F.pad(self.alphas_cumprod[:-1], (1, 0), value=1.0)
        
        # Calculations for diffusion q(x_t | x_0)
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - self.alphas_cumprod)
        
        # Calculations for posterior q(x_{t-1} | x_t, x_0)
        self.posterior_variance = betas * (1.0 - self.alphas_cumprod_prev) / (1.0 - self.alphas_cumprod)
        self.posterior_log_variance_clipped = torch.log(torch.clamp(self.posterior_variance, min=1e-20))
        self.posterior_mean_coef1 = betas * torch.sqrt(self.alphas_cumprod_prev) / (1.0 - self.alphas_cumprod)
        self.posterior_mean_coef2 = (1.0 - self.alphas_cumprod_prev) * torch.sqrt(self.alphas) / (1.0 - self.alphas_cumprod)
    
    def _cosine_beta_schedule(self, timesteps, s=0.008):
        """Cosine schedule from 'Improved DDPM'."""
        steps = timesteps + 1
        x = torch.linspace(0, timesteps, steps)
        alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * torch.pi * 0.5) ** 2
        alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
        betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
        return torch.clip(betas, 0.0001, 0.9999)
    
    def q_sample(self, x_0, t, noise=None):
        """Sample from q(x_t | x_0) - the forward diffusion process."""
        if noise is None:
            noise = torch.randn_like(x_0)
        
        sqrt_alphas_cumprod_t = self._extract(self.sqrt_alphas_cumprod, t, x_0.shape)
        sqrt_one_minus_alphas_cumprod_t = self._extract(self.sqrt_one_minus_alphas_cumprod, t, x_0.shape)
        
        return sqrt_alphas_cumprod_t * x_0 + sqrt_one_minus_alphas_cumprod_t * noise
    
    def _extract(self, a, t, x_shape):
        """Extract values from a at indices t."""
        batch_size = t.shape[0]
        out = a.gather(-1, t)
        return out.reshape(batch_size, *((1,) * (len(x_shape) - 1)))
    
    def to(self, device):
        """Move all tensors to device."""
        self.betas = self.betas.to(device)
        self.alphas = self.alphas.to(device)
        self.alphas_cumprod = self.alphas_cumprod.to(device)
        self.alphas_cumprod_prev = self.alphas_cumprod_prev.to(device)
        self.sqrt_alphas_cumprod = self.sqrt_alphas_cumprod.to(device)
        self.sqrt_one_minus_alphas_cumprod = self.sqrt_one_minus_alphas_cumprod.to(device)
        self.posterior_variance = self.posterior_variance.to(device)
        self.posterior_log_variance_clipped = self.posterior_log_variance_clipped.to(device)
        self.posterior_mean_coef1 = self.posterior_mean_coef1.to(device)
        self.posterior_mean_coef2 = self.posterior_mean_coef2.to(device)
        return self

## 2. U-Net Architecture for Time Series

We implement a 1D U-Net with time embeddings for noise prediction.

In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
    """Sinusoidal position embeddings for timestep encoding."""
    
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    
    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


class Block(nn.Module):
    """Convolutional block with GroupNorm and SiLU activation."""
    
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False):
        super().__init__()
        self.time_mlp = nn.Linear(time_emb_dim, out_ch)
        
        if up:
            self.conv1 = nn.Conv1d(2 * in_ch, out_ch, 3, padding=1)
            self.transform = nn.ConvTranspose1d(out_ch, out_ch, 4, 2, 1)
        else:
            self.conv1 = nn.Conv1d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.Conv1d(out_ch, out_ch, 4, 2, 1)
        
        self.conv2 = nn.Conv1d(out_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.GroupNorm(8, out_ch)
        self.bnorm2 = nn.GroupNorm(8, out_ch)
        self.relu = nn.SiLU()
    
    def forward(self, x, t):
        # First conv
        h = self.bnorm1(self.relu(self.conv1(x)))
        
        # Time embedding
        time_emb = self.relu(self.time_mlp(t))
        time_emb = time_emb[:, :, None]  # Extend to 1D
        h = h + time_emb
        
        # Second conv
        h = self.bnorm2(self.relu(self.conv2(h)))
        
        # Down or upsample
        return self.transform(h)


class UNet1D(nn.Module):
    """1D U-Net for time series diffusion."""
    
    def __init__(self, in_channels=1, out_channels=1, time_emb_dim=128, 
                 base_channels=64, channel_mults=(1, 2, 4)):
        super().__init__()
        
        # Time embedding
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.SiLU()
        )
        
        # Initial projection
        self.conv0 = nn.Conv1d(in_channels, base_channels, 3, padding=1)
        
        # Downsampling
        self.downs = nn.ModuleList()
        channels = [base_channels]
        in_ch = base_channels
        
        for mult in channel_mults:
            out_ch = base_channels * mult
            self.downs.append(Block(in_ch, out_ch, time_emb_dim))
            channels.append(out_ch)
            in_ch = out_ch
        
        # Middle
        self.mid1 = nn.Conv1d(in_ch, in_ch, 3, padding=1)
        self.mid2 = nn.Conv1d(in_ch, in_ch, 3, padding=1)
        
        # Upsampling
        self.ups = nn.ModuleList()
        for mult in reversed(channel_mults):
            out_ch = base_channels * mult
            self.ups.append(Block(in_ch, out_ch, time_emb_dim, up=True))
            in_ch = out_ch
        
        # Output
        self.output = nn.Conv1d(base_channels, out_channels, 1)
    
    def forward(self, x, t):
        # Time embedding
        t = self.time_mlp(t)
        
        # Initial conv
        x = self.conv0(x)
        
        # Downsample
        residuals = [x]
        for down in self.downs:
            x = down(x, t)
            residuals.append(x)
        
        # Middle
        x = F.silu(self.mid1(x))
        x = F.silu(self.mid2(x))
        
        # Upsample
        for up in self.ups:
            residual = residuals.pop()
            # Handle size mismatch
            if x.shape[-1] != residual.shape[-1]:
                x = F.interpolate(x, size=residual.shape[-1], mode='nearest')
            x = torch.cat((x, residual), dim=1)
            x = up(x, t)
        
        return self.output(x)

## 3. Simpler MLP-based Denoiser (for faster experimentation)

In [ ]:
class MLPDenoiser(nn.Module):
    """Simple MLP-based denoiser for 1D time series."""
    
    def __init__(self, seq_length, hidden_dim=256, time_emb_dim=64, num_layers=4):
        super().__init__()
        self.seq_length = seq_length
        
        # Time embedding
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_emb_dim),
            nn.Linear(time_emb_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Main network
        self.input_proj = nn.Linear(seq_length + hidden_dim, hidden_dim)
        
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.SiLU(),
                nn.Dropout(0.1)
            ))
        
        self.output_proj = nn.Linear(hidden_dim, seq_length)
    
    def forward(self, x, t):
        # x: [batch, seq_length] or [batch, 1, seq_length]
        if x.dim() == 3:
            x = x.squeeze(1)
        
        # Time embedding
        t_emb = self.time_mlp(t)
        
        # Concatenate input with time embedding
        h = torch.cat([x, t_emb], dim=-1)
        h = self.input_proj(h)
        
        # Apply layers with residual connections
        for layer in self.layers:
            h = h + layer(h)
        
        return self.output_proj(h)

## 4. Training

In [ ]:
def train_step(model, diffusion, optimizer, x_0):
    """Single training step."""
    batch_size = x_0.shape[0]
    
    # Sample random timesteps
    t = torch.randint(0, diffusion.num_timesteps, (batch_size,), device=x_0.device).long()
    
    # Sample noise
    noise = torch.randn_like(x_0)
    
    # Get noisy samples
    x_t = diffusion.q_sample(x_0, t, noise)
    
    # Predict noise
    noise_pred = model(x_t, t.float())
    
    # Ensure shapes match
    if noise_pred.dim() != noise.dim():
        if noise_pred.dim() == 2 and noise.dim() == 3:
            noise = noise.squeeze(1)
        elif noise_pred.dim() == 3 and noise.dim() == 2:
            noise_pred = noise_pred.squeeze(1)
    
    # Compute loss
    loss = F.mse_loss(noise_pred, noise)
    
    # Backprop
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    
    return loss.item()


def train(model, diffusion, dataloader, epochs=100, lr=1e-4):
    """Full training loop."""
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    
    losses = []
    
    for epoch in tqdm(range(epochs), desc="Training"):
        epoch_losses = []
        
        for batch in dataloader:
            x_0 = batch[0].to(device)
            loss = train_step(model, diffusion, optimizer, x_0)
            epoch_losses.append(loss)
        
        scheduler.step()
        avg_loss = np.mean(epoch_losses)
        losses.append(avg_loss)
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}: Loss = {avg_loss:.6f}")
    
    return losses

## 5. Sampling (DDPM and DDIM)

In [ ]:
@torch.no_grad()
def ddpm_sample(model, diffusion, shape, device):
    """Sample using DDPM (slow but high quality)."""
    model.eval()
    
    # Start from pure noise
    x = torch.randn(shape, device=device)
    
    for t in tqdm(reversed(range(diffusion.num_timesteps)), desc="DDPM Sampling", total=diffusion.num_timesteps):
        t_batch = torch.full((shape[0],), t, device=device, dtype=torch.long)
        
        # Predict noise
        noise_pred = model(x, t_batch.float())
        if noise_pred.dim() == 2:
            noise_pred = noise_pred.unsqueeze(1)
        
        # Get diffusion parameters
        alpha = diffusion.alphas[t]
        alpha_cumprod = diffusion.alphas_cumprod[t]
        beta = diffusion.betas[t]
        
        # Compute mean
        mean = (1 / torch.sqrt(alpha)) * (x - (beta / torch.sqrt(1 - alpha_cumprod)) * noise_pred)
        
        # Add noise (except for t=0)
        if t > 0:
            noise = torch.randn_like(x)
            sigma = torch.sqrt(beta)
            x = mean + sigma * noise
        else:
            x = mean
    
    return x


@torch.no_grad()
def ddim_sample(model, diffusion, shape, device, num_steps=50, eta=0.0):
    """
    Sample using DDIM (faster, controllable stochasticity).
    
    Args:
        eta: 0 = deterministic, 1 = DDPM-like stochasticity
    """
    model.eval()
    
    # Create timestep schedule (subset of full schedule)
    step_size = diffusion.num_timesteps // num_steps
    timesteps = list(range(0, diffusion.num_timesteps, step_size))
    timesteps = list(reversed(timesteps))
    
    # Start from pure noise
    x = torch.randn(shape, device=device)
    
    for i in tqdm(range(len(timesteps)), desc="DDIM Sampling"):
        t = timesteps[i]
        t_prev = timesteps[i + 1] if i < len(timesteps) - 1 else 0
        
        t_batch = torch.full((shape[0],), t, device=device, dtype=torch.long)
        
        # Predict noise
        noise_pred = model(x, t_batch.float())
        if noise_pred.dim() == 2:
            noise_pred = noise_pred.unsqueeze(1)
        
        # Get alpha values
        alpha_cumprod_t = diffusion.alphas_cumprod[t]
        alpha_cumprod_t_prev = diffusion.alphas_cumprod[t_prev] if t_prev >= 0 else torch.tensor(1.0)
        
        # Predict x_0
        x_0_pred = (x - torch.sqrt(1 - alpha_cumprod_t) * noise_pred) / torch.sqrt(alpha_cumprod_t)
        
        # Compute variance
        sigma = eta * torch.sqrt((1 - alpha_cumprod_t_prev) / (1 - alpha_cumprod_t)) * torch.sqrt(1 - alpha_cumprod_t / alpha_cumprod_t_prev)
        
        # Direction pointing to x_t
        dir_xt = torch.sqrt(1 - alpha_cumprod_t_prev - sigma**2) * noise_pred
        
        # Compute x_{t-1}
        x = torch.sqrt(alpha_cumprod_t_prev) * x_0_pred + dir_xt
        
        if t_prev > 0 and eta > 0:
            noise = torch.randn_like(x)
            x = x + sigma * noise
    
    return x

## 6. Generate Synthetic Financial Data and Train

In [ ]:
# Generate synthetic financial time series
def generate_synthetic_returns(n_samples=5000, seq_length=64):
    """Generate synthetic returns with realistic properties."""
    returns = np.zeros((n_samples, seq_length))
    
    for i in range(n_samples):
        # Base volatility
        base_vol = np.random.uniform(0.01, 0.03)
        
        # Generate with volatility clustering
        vol = base_vol
        for t in range(seq_length):
            if t > 0:
                # GARCH-like volatility
                vol = 0.9 * vol + 0.1 * base_vol * (1 + abs(returns[i, t-1]) / base_vol)
            returns[i, t] = vol * np.random.randn()
    
    return returns

# Generate data
seq_length = 64
returns = generate_synthetic_returns(n_samples=5000, seq_length=seq_length)

# Normalize
returns_mean = returns.mean()
returns_std = returns.std()
returns_normalized = (returns - returns_mean) / returns_std

# Create dataset
X = torch.tensor(returns_normalized, dtype=torch.float32)
dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f"Data shape: {X.shape}")
print(f"Mean: {returns.mean():.6f}, Std: {returns.std():.6f}")

In [ ]:
# Initialize model and diffusion
diffusion = GaussianDiffusion(num_timesteps=500, beta_schedule='cosine').to(device)
model = MLPDenoiser(seq_length=seq_length, hidden_dim=256, time_emb_dim=64, num_layers=4).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Train
losses = train(model, diffusion, dataloader, epochs=100, lr=1e-4)

# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Compare DDPM vs DDIM Sampling

In [ ]:
# Sample with both methods
n_samples = 100

# DDPM (slow but full quality)
print("Sampling with DDPM (500 steps)...")
ddpm_samples = ddpm_sample(model, diffusion, (n_samples, seq_length), device)

# DDIM (fast)
print("\nSampling with DDIM (50 steps)...")
ddim_samples = ddim_sample(model, diffusion, (n_samples, seq_length), device, num_steps=50, eta=0.0)

# Convert to numpy
ddpm_samples = ddpm_samples.cpu().numpy()
ddim_samples = ddim_samples.cpu().numpy()

# Denormalize
ddpm_samples = ddpm_samples * returns_std + returns_mean
ddim_samples = ddim_samples * returns_std + returns_mean

In [ ]:
# Compare distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Sample trajectories
for i in range(5):
    axes[0, 0].plot(np.cumsum(returns[i]), alpha=0.7)
axes[0, 0].set_title('Real Samples')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Cumulative Return')

for i in range(5):
    axes[0, 1].plot(np.cumsum(ddpm_samples[i]), alpha=0.7)
axes[0, 1].set_title('DDPM Samples (500 steps)')
axes[0, 1].set_xlabel('Time')

for i in range(5):
    axes[0, 2].plot(np.cumsum(ddim_samples[i]), alpha=0.7)
axes[0, 2].set_title('DDIM Samples (50 steps)')
axes[0, 2].set_xlabel('Time')

# Distribution comparison
axes[1, 0].hist(returns.flatten(), bins=100, density=True, alpha=0.7, label='Real')
axes[1, 0].hist(ddpm_samples.flatten(), bins=100, density=True, alpha=0.7, label='DDPM')
axes[1, 0].legend()
axes[1, 0].set_title('Distribution: Real vs DDPM')

axes[1, 1].hist(returns.flatten(), bins=100, density=True, alpha=0.7, label='Real')
axes[1, 1].hist(ddim_samples.flatten(), bins=100, density=True, alpha=0.7, label='DDIM')
axes[1, 1].legend()
axes[1, 1].set_title('Distribution: Real vs DDIM')

# QQ plot
real_sorted = np.sort(returns.flatten())
ddpm_sorted = np.sort(ddpm_samples.flatten())
ddim_sorted = np.sort(ddim_samples.flatten())

idx = np.linspace(0, len(real_sorted)-1, 1000).astype(int)
axes[1, 2].scatter(real_sorted[idx], ddpm_sorted[idx], alpha=0.3, s=10, label='DDPM')
axes[1, 2].scatter(real_sorted[idx], ddim_sorted[idx], alpha=0.3, s=10, label='DDIM')
axes[1, 2].plot([real_sorted.min(), real_sorted.max()], 
                [real_sorted.min(), real_sorted.max()], 'k--')
axes[1, 2].set_title('Q-Q Plot')
axes[1, 2].set_xlabel('Real Quantiles')
axes[1, 2].set_ylabel('Generated Quantiles')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 8. Visualize the Denoising Process

In [ ]:
@torch.no_grad()
def visualize_denoising(model, diffusion, device, num_viz_steps=10):
    """Visualize the denoising process."""
    model.eval()
    
    # Start from noise
    x = torch.randn(1, seq_length, device=device)
    
    # Store intermediate results
    step_size = diffusion.num_timesteps // num_viz_steps
    intermediates = [x.cpu().numpy().flatten()]
    timesteps_viz = [diffusion.num_timesteps]
    
    for t in reversed(range(diffusion.num_timesteps)):
        t_batch = torch.full((1,), t, device=device, dtype=torch.long)
        
        # Predict noise
        noise_pred = model(x, t_batch.float())
        if noise_pred.dim() == 2:
            noise_pred = noise_pred.unsqueeze(1)
        
        # Get parameters
        alpha = diffusion.alphas[t]
        alpha_cumprod = diffusion.alphas_cumprod[t]
        beta = diffusion.betas[t]
        
        # Denoise
        mean = (1 / torch.sqrt(alpha)) * (x - (beta / torch.sqrt(1 - alpha_cumprod)) * noise_pred)
        
        if t > 0:
            noise = torch.randn_like(x)
            x = mean + torch.sqrt(beta) * noise
        else:
            x = mean
        
        # Store intermediate
        if t % step_size == 0 or t == 0:
            intermediates.append(x.cpu().numpy().flatten())
            timesteps_viz.append(t)
    
    return intermediates, timesteps_viz

# Visualize
intermediates, timesteps_viz = visualize_denoising(model, diffusion, device)

fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for i, (sample, t) in enumerate(zip(intermediates[:10], timesteps_viz[:10])):
    # Denormalize
    sample_denorm = sample * returns_std + returns_mean
    axes[i].plot(np.cumsum(sample_denorm), 'b-', linewidth=1.5)
    axes[i].set_title(f't = {t}')
    axes[i].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    axes[i].set_xlabel('Time')
    if i % 5 == 0:
        axes[i].set_ylabel('Cumulative Return')

plt.suptitle('Denoising Process: From Noise to Financial Time Series', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Key Takeaways

1. **DDPM** learns to reverse a gradual noising process
2. **U-Net** architecture can be adapted for 1D time series
3. **DDIM** provides 10x faster sampling with minimal quality loss
4. **Noise schedule** (cosine vs linear) affects quality significantly
5. Generated samples capture distribution statistics well

### Next Steps:
- Notebook 03: TimeGrad for conditional forecasting
- Notebook 04: CSDI for imputation and forecasting